In [1]:
import numpy as np
import random
import math

def dataloader(filename, mean = None, std = None, split="train"):
    with open(filename) as f:
        all_data = f.readlines()
        np_data = []
        for each_data in all_data:
            per_elem = []
            for col in each_data.split():
                per_elem.append(float(col))
            np_data.append(per_elem)
                
        np_data = np.array(np_data)
        features = np_data[:, :-1]
        
        if mean is None:
            mean, std = features.mean((0)), features.std((0))
        
        features = (features - mean) / std
        
        labels = np_data[:, -1] - 1
        unique_labels = np.unique(labels)
        
        if split == "train":
            # one-hot creation
           
            one_hot_label_vector = []
            
            for each_data in labels:
                y_actual = np.zeros([unique_labels.shape[0]])
                y_actual[int(each_data)] = 1   
                one_hot_label_vector.append(y_actual)

            
            return features, one_hot_label_vector, features.shape[1], unique_labels.shape[0], features.shape[0], mean, std
        else:
            return features, labels, features.shape[0]

In [2]:
class NN:
    
    def __init__(self, layer_num, node_num, feature_num, label_num, mu=0.9, alpha=1.):
        self.alpha, self.mu, self.feature_num, self.label_num = alpha, mu, feature_num, label_num
        self.layer_num = layer_num + 1
        self.f_x = np.vectorize(self.logit) # logit
        
        self.weight = []
        self.weight.append(np.random.rand(self.feature_num+1, node_num))
        
        for i in range(layer_num-1):
            self.weight.append(np.random.rand(node_num+1, node_num)) # weight matrix of size nodes_in_prev_layer+1 * next_layer
        self.weight.append(np.random.rand(node_num+1, self.label_num))
        
        if ((layer_num+1) != len(self.weight)):
            print("error!")
            exit()              
       
    
    def logit(self, x):
        return 1 / (1 + math.exp(-1 * self.alpha * x)) # eq 4.1   
    
    def f_hat_x(self, f_x):
        return self.alpha * f_x * (1 - f_x) 
    
    def forward(self, x):
        
        self.y = []        
        y_k = np.ones([self.feature_num+1])
        y_k[:-1] = x
        
        for w_k in self.weight:#eqn4.7
            v_r = np.dot(w_k.T, y_k)
            y_k = np.ones([w_k.shape[1]+1])
            y_k[:-1] = self.f_x(v_r)
            self.y.append(y_k[:-1])
    
    def backward(self, y_actual):
        e_k = self.y[-1] - y_actual #eq: 4.14
        self.delta_j = []
        
        for k in reversed(list(range(self.layer_num))):
            
            delta_k = e_k * self.f_hat_x(self.y[k]) #eqn 4.22
            self.delta_j.append(np.copy(delta_k))
            
            w_k = self.weight[k]
            e_k = np.dot(w_k, delta_k)[:-1] #eqn 4.23
            
        self.delta_j.reverse()
        
    def update(self, x):
        
        self.del_wj = [np.zeros(self.weight[r].shape) for r in range(self.layer_num)]        
        y_r_prev = np.ones([self.feature_num+1])
        y_r_prev[:-1] = x
        
        for i in range(self.layer_num): 
            self.del_wj[i] = self.del_wj[i] + np.dot(y_r_prev[..., np.newaxis], self.delta_j[i][np.newaxis, ...]) 
            self.weight[i] = self.weight[i] - self.mu * self.del_wj[i] 
            
            y_r_prev = np.ones([self.weight[i].shape[1] + 1])
            y_r_prev[:-1] = self.y[i]
             
    

In [3]:
def Training_Helper(NN, all_train_features, all_train_labels):

    for feature, y_actual in zip(all_train_features, all_train_labels):
        NN.forward(x=feature)
        NN.backward(y_actual=y_actual)
        NN.update(x=feature)


def Test_Helper(NN, all_test_features, all_test_labels):
    final_str = ""
    total = all_test_features.shape[0]
    correct = 0
    
    for iter, (feature, y_actual) in enumerate(zip(all_test_features, all_test_labels)):
        
        NN.forward(x=feature)
        y_pred = (NN.y[-1]).argmax()
        
        if int(y_actual) == y_pred:
            correct += 1
        else:
            final_str += "sample no:" + str(iter) + " actual class:" +str(y_actual) + " predicted class:" + str(y_pred) + '\n'
        
    return final_str, 100 * correct / total

In [4]:
features_train, train_label_vector, feature_num, label_num, total_samples_train, mean, std = dataloader(filename='trainNN.txt')
NN_callable = NN(layer_num=2, node_num=5, feature_num=feature_num, label_num=label_num)

epoch_no = 100
features_test, labels_test, total_samples_test = dataloader(filename='testNN.txt', mean=mean, std=std, split="test")

best_accuracy = 0
for i in range(epoch_no):
    
    Training_Helper(NN_callable, all_train_features=features_train, all_train_labels=train_label_vector)
    _, test_accuracy = Test_Helper(NN_callable, all_test_features=features_test, all_test_labels=labels_test)
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy    
    
    if best_accuracy == 100.0:
        break
    
print("Total Accuracy:", best_accuracy,"%")    


Total Accuracy: 100.0 %


In [5]:
error_report, accuracy = Test_Helper(NN_callable, all_test_features =features_test, all_test_labels=labels_test)
print(error_report)

file_handle = open("error.txt", "w")
file_handle.write(error_report)
file_handle.close()

In [6]:
features_train, train_label_vector, feature_num, label_num, total_samples_train, mean, std = dataloader(filename='trainNNeval.txt')
features_test, labels_test, total_samples_test = dataloader(filename='testNNeval.txt', mean=mean, std=std, split="test")


layer=(np.random.randint(1,5, size=(10)))
nodes=(np.random.randint(1,10, size=(10)))

report_str = " "
for each_comb in zip(layer,nodes):
        
    NN_callable = NN(layer_num=each_comb[0], node_num=each_comb[1], feature_num=feature_num, label_num=label_num)

    epoch_no = 100

    best_accuracy = 0
    for i in range(epoch_no):
        Training_Helper(NN_callable, all_train_features=features_train, all_train_labels=train_label_vector)
        _, test_accuracy = Test_Helper(NN_callable, all_test_features=features_test, all_test_labels=labels_test)
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy    

        if best_accuracy == 100.0:
            break
            
#     print("Total Accuracy:", best_accuracy,"%")    

    _, accuracy = Test_Helper(NN_callable, all_test_features =features_test, all_test_labels=labels_test)
    print("no. of layers:",each_comb[0], "no. of nodes/layer:",each_comb[1] ,"accuracy:",accuracy) 
    report_str += '\n' + str(each_comb[0]) + "    " + str(each_comb[1]) + "    " + str(accuracy) 
file_handle = open("1505052.txt", "w")
file_handle.write(report_str)
file_handle.close()

no. of layers: 2 no. of nodes/layer: 9 accuracy: 100.0
no. of layers: 2 no. of nodes/layer: 7 accuracy: 100.0
no. of layers: 4 no. of nodes/layer: 7 accuracy: 100.0
no. of layers: 1 no. of nodes/layer: 7 accuracy: 100.0
no. of layers: 2 no. of nodes/layer: 2 accuracy: 100.0
no. of layers: 3 no. of nodes/layer: 7 accuracy: 100.0
no. of layers: 3 no. of nodes/layer: 4 accuracy: 100.0
no. of layers: 4 no. of nodes/layer: 1 accuracy: 31.4
no. of layers: 3 no. of nodes/layer: 8 accuracy: 100.0
no. of layers: 4 no. of nodes/layer: 9 accuracy: 100.0
